In [1]:
# TripAdvisor Reviews
import requests
from bs4 import BeautifulSoup
import time
import random
import re
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

In [72]:
site_category_list = ['https://www.tripadvisor.com.ph/Attractions-g293790-Activities-c47-t2,3,10,15,17,19,26,76,163,175-Ethiopia.html', # Sights and Landmarks
                    'https://www.tripadvisor.com.ph/Attractions-g293790-Activities-c57-t57,66,67,68,70-Ethiopia.html', # Nature and Parks
                     'https://www.tripadvisor.com.ph/Attractions-g293790-Activities-c49-t1,28,29,30,161-Ethiopia.html'] # Museums
skipped_urls = []
first_page_list = []
# first_page_list = []

In [73]:
def getFirstPages(x):
    for i in x:
        page = requests.get(i)
        soup = BeautifulSoup(page.content, 'html.parser')
        sites = soup.find_all(class_ = 'attraction_element')
        for j in sites:
            if j.find(class_ = 'rs rating') != None:
                first_page_list.append('https://www.tripadvisor.com.ph/' + j.find(class_ = 'listing_title').a.get('href'))
        if len(sites) == 0 and i not in skipped_urls:
            skipped_urls.append(i)
        if len(sites) > 0 and i in skipped_urls:
            skipped_urls.remove(i)
        print('skipped: ', skipped_urls)
        time.sleep(random.uniform(15, 30))
    if len(skipped_urls) != 0:
        print('restarting function...')
        getFirstPages(skipped_urls)

In [74]:
getFirstPages(site_category_list)

skipped:  []
skipped:  []
skipped:  []


In [82]:
string = 'Test'
string2 = string[0:1] + string[2:]
print(string2)

Tst


In [70]:
import collections
print(collections.Counter(first_page_list) == collections.Counter(list(set(first_page_list))))
print(len(first_page_list))

True
['https://www.tripadvisor.com.ph//Attraction_Review-g480193-d10147730-Reviews-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph//Attraction_Review-g480193-d324957-Reviews-Rock_Hewn_Churches_of_Lalibela-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph//Attraction_Review-g2214567-d5966995-Reviews-Abuna_Yemata_Guh_church-Hawzen_Tigray_Region.html', 'https://www.tripadvisor.com.ph//Attraction_Review-g480193-d5427774-Reviews-Yemrehanna_Kristos-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph//Attraction_Review-g317059-d4102519-Reviews-Debre_Berhan_Selassie_Church-Gonder_Amhara_Region.html', 'https://www.tripadvisor.com.ph//Attraction_Review-g480192-d7789536-Reviews-Azwa_Mariam_Monastery-Bahir_Dar_Amhara_Region.html', 'https://www.tripadvisor.com.ph//Attraction_Review-g676757-d6620573-Reviews-Harar_Jegol_Wall-Harar_Harari_Region.html', 'https://www.tripadvisor.com.ph//Attraction_Review-g317059-d2540370-Reviews-Fasiladas_Bath_Timk

In [78]:
print(len(first_page_list))

78


In [20]:
rev_place_list = []
rev_head_list = []
rev_body_list = []
rev_rating_list = []
rev_date_list = []
page_nos = []
skipped_urls = []
url_list1 = []

In [21]:
url_list1 = ['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d324957-Reviews-or710-Rock_Hewn_Churches_of_Lalibela-Lalibela_Amhara_Region.html',
            'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d324957-Reviews-or720-Rock_Hewn_Churches_of_Lalibela-Lalibela_Amhara_Region.html']

In [22]:
# New
def getPlace(x, y):
    for i in y.find_all(class_ = 'noQuotes'):
        rev_place_list.append(x.find(id = 'HEADING').string)
        
def getHead(x):
    for i in x.find_all(class_ = 'noQuotes'):
        rev_head_list.append(i.string)

def getBody(x):
    z = x.find_all(class_ = 'entry')
    for i in z:
        if i.find(class_ = 'taLnk ulBlueLinks') != None:
            rev_body_list.append(i.text[:-4])
        else:
            rev_body_list.append(i.text)

def getRating(x):
    rev_revs = x.find_all(class_='ui_column is-9')
    for i in range(len(rev_revs)):
        if rev_revs[i].find(class_ = 'ui_bubble_rating bubble_50') != None:
            rev_rating_list.append(5)
        elif rev_revs[i].find(class_ = 'ui_bubble_rating bubble_40') != None:
            rev_rating_list.append(4)
        elif rev_revs[i].find(class_ = 'ui_bubble_rating bubble_30') != None:
            rev_rating_list.append(3)
        elif rev_revs[i].find(class_ = 'ui_bubble_rating bubble_20') != None:
            rev_rating_list.append(2)
        elif rev_revs[i].find(class_ = 'ui_bubble_rating bubble_10') != None:
            rev_rating_list.append(1)

def getDate(x):
    for i in x.find_all(class_ = 'prw_rup prw_reviews_stay_date_hsx'):
        if i.find(class_ = 'stay_date_label') == None:
            rev_date_list.append('None')
        else:
            rev_date_list.append(str(i.span.next_sibling).strip())

def delay(x, y):
    time.sleep(random.uniform(x, y))
    return None

def getData(urls):
    start_revs = 0
    end_revs = 0
    for i in urls:
        start_revs = len(rev_head_list)
        print('start: ', len(rev_head_list))
        page = requests.get(i, headers = headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        rev = soup.find(id = 'REVIEWS')
        getPlace(soup, rev)
        getHead(rev)
        getBody(rev)
        getRating(rev)
        getDate(rev)
        delay(15, 30)
        end_revs = len(rev_head_list)
        print('end: ', len(rev_head_list))
        if end_revs == start_revs and i not in skipped_urls:
            skipped_urls.append(i)
        if end_revs > start_revs and i in skipped_urls:
            skipped_urls.remove(i)
        print(skipped_urls)
    if len(skipped_urls) != 0:
        print('restarting function...')
        getData(skipped_urls)

In [23]:
getData(url_list1)

start:  0
end:  10
[]
start:  10
end:  20
[]


In [24]:
print(rev_place_list)
print(rev_head_list)
print(rev_body_list)
print(rev_date_list)
print(rev_rating_list)

['Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela', 'Rock-Hewn Churches of Lalibela']
['Amazing and impressing', 'Amazing churches of LAlibela', "Couldn't help but to compare to Petra!", "It's the one you see in the books!", 'Magical', 'Amazing churches', 'Unique churches', 'Just like the pictures', 'A cross shaped church is just the beginning', 'Amazing', 'the strength of man', 'Great Site- Marred By 

In [25]:
print(len(rev_place_list))
print(len(rev_head_list))
print(len(rev_body_list))
print(len(rev_rating_list))
print(len(rev_date_list))

20
20
20
20
20


In [ ]:
https://www.tripadvisor.com.ph/Attraction_Review-g480193-d324957-Reviews-or720-Rock_Hewn_Churches_of_Lalibela-Lalibela_Amhara_Region.html

In [19]:
page = requests.get('https://www.tripadvisor.com.ph/Attraction_Review-g480193-d324957-Reviews-or710-Rock_Hewn_Churches_of_Lalibela-Lalibela_Amhara_Region.html', headers = headers)
soup = BeautifulSoup(page.content, 'html.parser')
rev = soup.find(id = 'REVIEWS')
for i in rev.find_all(class_ = 'prw_rup prw_reviews_stay_date_hsx'):
    print('1')
    print(i.find(class_ = 'stay_date_label'))
    if i.find(class_ = 'stay_date_label') == None:
        rev_date_list.append('None')
    else:
        rev_date_list.append(str(i.span.next_sibling).strip())

1
<span class="stay_date_label">Date of experience:</span>
1
<span class="stay_date_label">Date of experience:</span>
1
None
1
None
1
<span class="stay_date_label">Date of experience:</span>
1
<span class="stay_date_label">Date of experience:</span>
1
<span class="stay_date_label">Date of experience:</span>
1
<span class="stay_date_label">Date of experience:</span>
1
<span class="stay_date_label">Date of experience:</span>
1
<span class="stay_date_label">Date of experience:</span>
